In [ ]:
pip install -U bitsandbytes transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 123.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.3
    Uninstalling transformers-4.57.3:
      Successfully uninstalled transformers-4.57.3


In [ ]:
from huggingface_hub import login
login()


# imports & Settings

In [ ]:
import torch
import pandas as pd
from datasets import Dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)

MODEL_ID = "meta-llama/Llama-3.2-1B"
REPO_NAME = "Tushar1K/llama3.2-qlora-sentiment"


# Dataset Loading

In [ ]:
def parse_txt(file_path):
    texts, labels = [], []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            label, text = line.split(" ", 1)
            texts.append(text.strip())
            labels.append(int(label.replace("__label__", "")) - 1)
    return pd.DataFrame({"text": texts, "label": labels})

train_ds = Dataset.from_pandas(parse_txt("train.ft.txt"))
test_ds  = Dataset.from_pandas(parse_txt("test.ft.txt"))


In [ ]:
len(train_ds), len(test_ds)

(268561, 265867)

In [ ]:
print(train_ds[0])
print(test_ds[0])

{'text': 'Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^', 'label': 1}
{'text': 'Great CD: My lovely Pat has one of the GREAT voices of her generation. I have listened to this CD for YEARS and I still LOVE IT. When I\'m in a good mood it makes me feel better. A bad mood just evaporates like sugar in the rain. This CD just oozes LIFE. Vocals are jusat STUUNNING and lyrics just kill. One of life\'s hidden gems. This is a desert isle CD in my book. Why she never made it big is just beyond me. Everytime I play this, no matter black, white, young, old, male, female EVERYBODY says one thing "Who was that singing ?"', '

In [ ]:
print(train_ds[1])
print(test_ds[1])

{'text': "The best soundtrack ever to anything.: I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This in my opinino is Yasunori Mitsuda's ultimate masterpiece. The music is timeless and I'm been listening to it for years now and its beauty simply refuses to fade.The price tag on this is pretty staggering I must say, but if you are going to buy any cd for this much money, this is the only one that I feel would be worth every penny.", 'label': 1}
{'text': "One of the best game music soundtracks - for a game I didn't really play: Despite the fact that I have only played a small portion of the game, the music I heard (plus the connection to Chrono Trigger which was great as well) led me to purchase the soundtrack, and it remains one of my favorite albums. There is an incredible mix of fun, epic, and emotional songs. Those sad and beautiful tracks I especially like, as there's not too many of those kinds of

In [ ]:
print(test_ds[-1])
print(train_ds[-1])


{'text': "it's now a paperweight: Please note that this mic does not connect directly to a standard XLR mic cable. It has an odd 6-pin cable that connects it to its power supply. (The power supply box DOES have XLR out.)My problem is that the 6-pin cable wore out rather quickly, and finding a replacement is rather difficult. No one at the music supply stores know what I'm talking about when I ask for itI am not sure why this mic couldn't have been powered with a standard XLR cable. Whatever the reason, it has turned my mic into a paperweight.From what I remember, this mic sounded good, but it has be", 'label': 0}
{'text': "Always an inspirational read ... first time or twentieth!: I first read Karen when I was in grade school. Since then, I've worn out at least four copies with numerous re-reads. The sequel, With Love From Karen, is equally inspirational ... if only there w", 'label': 1}


# Quantization + Model

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_ID,
    num_labels=2,
    quantization_config=bnb_config,
    device_map="auto",
    token=True
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# QLoRA

In [ ]:
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,                      
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 856,064 || all params: 1,236,674,560 || trainable%: 0.0692


# Tokenization

In [ ]:
def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        max_length=256   
    )

train_ds = train_ds.map(tokenize, batched=True, remove_columns=["text"])
test_ds  = test_ds.map(tokenize, batched=True, remove_columns=["text"])


Map:   0%|          | 0/268561 [00:00<?, ? examples/s]

Map:   0%|          | 0/265867 [00:00<?, ? examples/s]

# Training Arguments

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    max_steps=2000,
    logging_steps=25,
    save_steps=500,
    save_total_limit=2,
    fp16=True,
    eval_strategy="no",
    report_to="none",
    push_to_hub=True,
    hub_model_id=REPO_NAME,
)


# Trainer

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
)

trainer.train()
trainer.push_to_hub()


/tmp/ipython-input-2695413154.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 128001}.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return f

Step,Training Loss
25,3.245200
50,1.132700
75,1.207500
100,0.772700
125,0.677400
150,0.712200
175,0.997500
200,0.850300
225,0.572100
250,0.688600


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/pyt

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...results/training_args.bin: 100%|##########| 5.84kB / 5.84kB            

  ...adapter_model.safetensors: 100%|##########| 3.43MB / 3.43MB            

  ...nt/results/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            

CommitInfo(commit_url='https://huggingface.co/Tushar1K/llama3.2-qlora-sentiment/commit/355e778624e4289718c0ff3829e301da9ff8d549', commit_message='End of training', commit_description='', oid='355e778624e4289718c0ff3829e301da9ff8d549', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Tushar1K/llama3.2-qlora-sentiment', endpoint='https://huggingface.co', repo_type='model', repo_id='Tushar1K/llama3.2-qlora-sentiment'), pr_revision=None, pr_num=None)

# EVALUATE ACCURACY

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds)
    }


In [16]:
trainer.compute_metrics = compute_metrics
metrics = trainer.evaluate(eval_dataset=test_ds)
print(metrics)


{'eval_loss': 0.14172537624835968, 'eval_accuracy': 0.967453651637849, 'eval_f1': 0.9673637958021385, 'eval_runtime': 7340.5125, 'eval_samples_per_second': 36.219, 'eval_steps_per_second': 4.527, 'epoch': 0.11915223187024322}


# INFERENCE

In [17]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=256).to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
    pred = torch.argmax(outputs.logits, dim=-1).item()
    return "Positive" if pred == 1 else "Negative"

print(predict_sentiment("This product is absolutely amazing"))
print(predict_sentiment("Worst experience ever, total waste of money"))


Positive
Negative


In [18]:
texts = [
    "Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen!",
    "One of the best game music soundtracks - for a game I didn't really play: Despite the fact that I have only played a small portion of the game, the music I heard (plus the connection to Chrono Trigger which was great as well) led me to purchase the soundtrack, and it remains one of my favorite albums. There is an incredible mix of fun, epic, and emotional songs. Those sad and beautiful tracks I especially like, as there's not too many of those kinds of songs in my other video game soundtracks. I must admit that one of the songs (Life-A Distant Promise) has brought tears to my eyes on many occasions.My one complaint about this soundtrack is that they use guitar fretting effects in many of the songs, which I find distracting. But even if those weren't included I would still consider the collection worth it."
]

inputs = tokenizer(texts, padding=True, truncation=True, max_length=256, return_tensors="pt").to(model.device)

with torch.no_grad():
    logits = model(**inputs).logits

preds = torch.argmax(logits, dim=1)
labels = ["Negative", "Positive"]

for t, p in zip(texts, preds):
    print(t, "→", labels[p])


Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! → Positive
One of the best game music soundtracks - for a game I didn't really play: Despite the fact that I have only played a small portion of the game, the music I heard (plus the connection to Chrono Trigger which was great as well) led me to purchase the soundtrack, and it remains one of my favorite albums. There is an incredible mix of fun, epic, and emotional songs. Those sad and beautiful tracks I especially like, as there's not too many of those kinds of songs in my other video game soundtracks. I must admit that one of the songs (Life-A Distant Promise) has bro


# MERGE LoRA

In [19]:
from peft import PeftModel

merged_model = model.merge_and_unload()

merged_model.save_pretrained("merged_llama_sentiment")
tokenizer.save_pretrained("merged_llama_sentiment")


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


('merged_llama_sentiment/tokenizer_config.json',
 'merged_llama_sentiment/special_tokens_map.json',
 'merged_llama_sentiment/tokenizer.json')

In [20]:
merged_model.push_to_hub("Tushar1K/llama3.2-merged-sentiment")
tokenizer.push_to_hub("Tushar1K/llama3.2-merged-sentiment")


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...lrd36oh/model.safetensors:   3%|2         | 41.6MB / 1.55GB            

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mpkjlnk3c2/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            

CommitInfo(commit_url='https://huggingface.co/Tushar1K/llama3.2-merged-sentiment/commit/b706223bf0f7bf4fd669c346d61a5e95911ad3d3', commit_message='Upload tokenizer', commit_description='', oid='b706223bf0f7bf4fd669c346d61a5e95911ad3d3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Tushar1K/llama3.2-merged-sentiment', endpoint='https://huggingface.co', repo_type='model', repo_id='Tushar1K/llama3.2-merged-sentiment'), pr_revision=None, pr_num=None)

# Convert to GGUF (Ollama)

# Export to FP16 (for Ollama)

In [21]:
merged_model.save_pretrained("fp16_model", safe_serialization=True)
tokenizer.save_pretrained("fp16_model")


('fp16_model/tokenizer_config.json',
 'fp16_model/special_tokens_map.json',
 'fp16_model/tokenizer.json')